## 1. 뉴스제목 가져오기
* user-agent 요청헤더를 반드시 설정해야 한다.

In [4]:
# requests 라이브러리 설치여부 확인
!pip show requests

Name: requests
Version: 2.32.3
Summary: Python HTTP for Humans.
Home-page: https://requests.readthedocs.io
Author: Kenneth Reitz
Author-email: me@kennethreitz.org
License: Apache-2.0
Location: C:\Users\user\anaconda3\Lib\site-packages
Requires: certifi, charset-normalizer, idna, urllib3
Required-by: anaconda-catalogs, anaconda-client, anaconda-cloud-auth, anaconda-project, conda, conda-build, conda-repo-cli, conda_package_streaming, cookiecutter, datashader, jupyterlab_server, panel, requests-file, requests-toolbelt, Sphinx, streamlit, tldextract


In [5]:
# beautifulsoup4 라이브러리 설치여부 확인
!pip show beautifulsoup4

Name: beautifulsoup4
Version: 4.12.3
Summary: Screen-scraping library
Home-page: https://www.crummy.com/software/BeautifulSoup/bs4/
Author: 
Author-email: Leonard Richardson <leonardr@segfault.org>
License: MIT License
Location: C:\Users\user\anaconda3\Lib\site-packages
Requires: soupsieve
Required-by: conda-build, nbconvert


In [6]:
# reqeusts, bs4 import
import requests 
# BeautifulSoup import
import bs4
#beautifulsoup 클래스 import
from bs4 import BeautifulSoup

In [7]:
# requests, bs4 버전 확인하기
print(f'requests 버전 = {requests.__version__}')
print(f'bs4 버전 = {bs4.__version__}')

requests 버전 = 2.32.3
bs4 버전 = 4.12.3


### 1. 뉴스 제목 추출하기

In [8]:
# IT/과학 뉴스 
req_param = {
    'sid': 105
}

sid = 104
#url =f'https://news.naver.com/section/{sid}' 
url = 'https://news.naver.com/section/{sid}'.format(**req_param)
print(url)


https://news.naver.com/section/105


In [9]:
# IT/과학 뉴스 
req_param = {
    'sid': 105
}
url = 'https://news.naver.com/section/{sid}'.format(**req_param)
print(url)

# 요청 헤더 설정 : 브라우저 정보
req_header = {
    "user-agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/104.0.0.0 Safari/537.36"
}

# requests 의 get() 함수 호출하기 
res=requests.get(url,headers=req_header)
print(res.status_code, res.ok)
print(type(res))
print(res)

# 응답(response)이 OK 이면
# 응답 (response)에서 text 추출
if res.ok:
    html = res.text
    # BeautifulSoup 객체 생성
    soup = BeautifulSoup(html, "html.parser")   
    #print(soup)
    # CSS 선택자
    # print(soup.select("div.sa_text a[href*='mnews/article']"))
    '''
       <div class='sa_text'> 
          <a href="https://n.news.naver.com/mnews/article/366/0001094461" class="sa_text_title">
    '''
    a_tags = soup.select("div.sa_text a[href*='https://n.news.naver.com/mnews/article'].sa_text_title")
    print(len(a_tags))
    print(type(a_tags)) #[Tag,Tag,Tag]
    # <a> 태그 리스트 순회하기
    for idx,a_tag in enumerate(a_tags,1):        
        if a_tag.text:
            title = a_tag.text.strip() # 제목
            link = a_tag['href'] # 링크
            #print(len(title))
            print(f'{idx} {title} {link}')
else:
    # 응답(response)이 Error 이면 status code 출력    
    print(f'Error Code = {res.status_code}')


https://news.naver.com/section/105
200 True
<class 'requests.models.Response'>
<Response [200]>
43
<class 'bs4.element.ResultSet'>
1 솔트룩스, KT ‘믿음’과 손잡아…‘독자 AI’ 사업 참전 https://n.news.naver.com/mnews/article/009/0005529262
2 갤럭시 Z 폴드7·Z 플립7, 폴더블 최다 사전판매 신기록.. 개통 시작 https://n.news.naver.com/mnews/article/014/0005380605
3 네이버클라우드, 140억 파라미터 추론모델도 상업용 무료 공개 https://n.news.naver.com/mnews/article/277/0005626112
4 불법 사이트 수사 피해 日 귀화까지…만화가들 "뉴토끼 운영자 송환 촉구" https://n.news.naver.com/mnews/article/119/0002982265
5 "올해 삼성 폴더블폰 점유율 10%P↓ 전망…내년 애플 출시 관건" https://n.news.naver.com/mnews/article/001/0015522990
6 ‘K-AI휴머노이드’ 개발…출연연·대학·기업 어벤져스 ‘전략연구단’ 떴다 https://n.news.naver.com/mnews/article/016/0002503305
7 게임, K콘텐츠 주역인데…일자리 8만개 사라질 수도 있다고? https://n.news.naver.com/mnews/article/011/0004512256
8 "SKT 위약금 면제로 7조 원 손실? 국민 협박, 사과 안 하면 고발" https://n.news.naver.com/mnews/article/006/0000130992
9 “50% 할인에도 하루 1대도 안팔려” 망할 줄 알았더니… 난리난 ‘접는폰’ 반전 https://n.news.naver.com/mnews/article/016/0002503447
10 한화에어로-항우연, 누

### 1.1 뉴스제목 추출하는 함수 선언하기

In [10]:
import requests
from bs4 import BeautifulSoup

#section_dict = {100:'정치',101:'경제',102:'사회',103:'생활/문화',104:'세계',105:'IT/과학'}
section_dict = {'정치':100,'경제':101,'사회':102,'생활/문화':103,'세계':104,'IT/과학':105}

def print_news(section):  #print_new(103,'생활/문화') 
    sid = section_dict[section]
    url = f'https://news.naver.com/section/{sid}'
    print(f'{url} {section} 뉴스')

    # 요청 헤더 설정 : 브라우저 정보
    req_header = {
        "user-agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/104.0.0.0 Safari/537.36"
    }
    # requests 의 get() 함수 호출하기 
    res=requests.get(url,headers=req_header)
    print(res.status_code, res.ok)
    if res.ok:
        html = res.text
        # BeautifulSoup 객체 생성
        soup = BeautifulSoup(html, "html.parser")
        a_tags = soup.select("div.sa_text a[href*='https://n.news.naver.com/mnews/article'].sa_text_title")
        print(len(a_tags))
        # <a> 태그 리스트 순회하기
        for idx,a_tag in enumerate(a_tags,1):        
            if a_tag.text:
                title = a_tag.text.strip() # 제목
                link = a_tag['href'] # 링크
                #print(len(title))
                print(f'{idx} {title} {link}')
    else:
        # 응답(response)이 Error 이면 status code 출력    
        print(f'Error Code = {res.status_code}')      

   

In [11]:
print_news('경제')

https://news.naver.com/section/101 경제 뉴스
200 True
46
1 프리즘, 제4회 청룡시리즈어워즈 생중계…"전 세계 325만명 시청" https://n.news.naver.com/mnews/article/138/0002201095
2 조종사노조 "항철위의 조종사 과실 프레임, 법적대응 검토" https://n.news.naver.com/mnews/article/422/0000762597
3 LG '엑사원' B2B 생태계 만든다…"AI 글로벌 톱3 도약할 것" https://n.news.naver.com/mnews/article/011/0004512227
4 여름 휴가 정점 8월 2, 3일 서울~부산·부산~서울 각각  6시간 10분 걸릴 듯 https://n.news.naver.com/mnews/article/658/0000114373
5 대기업 M&A 성과 1위는?…매출 140% 늘어난 '이곳' https://n.news.naver.com/mnews/article/011/0004512171
6 코스피 장중 연고점 찍고 하락…코스닥도 하락 마감 https://n.news.naver.com/mnews/article/056/0011993890
7 한미, 25일 미국서 ‘2+2’ 통상협의…막판 총력전 https://n.news.naver.com/mnews/article/056/0011993755
8 하나證, 퇴직연금 DB형 원리금 보장 상품 2분기 수익률 1위 https://n.news.naver.com/mnews/article/011/0004512120
9 한화에어로, 누리호 기술 이전받는다…25일 항우연과 체결식 https://n.news.naver.com/mnews/article/032/0003384593
10 코로나부터 반도체·관세 파고까지...현대차는 어떻게 버텼나 https://n.news.naver.com/mnews/article/277/0005625841
11 "직장인이 비싼 장어덮밥 어떻게 먹나요"…여기선 1만원도 안 

### 2. Image 다운로드
* referer 요청 헤더를 반드시 설정해야 한다.

In [ ]:
import requests
import os

req_header = {
    'referer':'https://comic.naver.com/webtoon/detail?titleId=783053&no=194&week=tue'      
}

img_urls = [
    'https://image-comic.pstatic.net/webtoon/783053/194/20250609204952_371810a746fd64e5cab0a6f546e5e690_IMAG01_30.jpg',
    'https://image-comic.pstatic.net/webtoon/783053/194/20250609204952_371810a746fd64e5cab0a6f546e5e690_IMAG01_31.jpg',
    'https://image-comic.pstatic.net/webtoon/783053/194/20250609204952_371810a746fd64e5cab0a6f546e5e690_IMAG01_32.jpg'
]

for img_url in img_urls:
    # requests 의 get(url, headers) 함수 호출하기 
    res = requests.get(img_url,headers=req_header)
    print(res.ok, res.status_code) 
    if res.ok:      
        # binary 응답 데이터 가져오기
        img_data = res.content
        # url에서 파일명만 추출하기
        file_name = os.path.basename(img_url)
        # binday data를 file에 write하기
        with open(file_name, 'wb') as file:
            print(f'Writing to {file_name}({len(img_data)})')
            file.write(img_data)


True 200
20250609204952_371810a746fd64e5cab0a6f546e5e690_IMAG01_30.jpg
Writing to 20250609204952_371810a746fd64e5cab0a6f546e5e690_IMAG01_30.jpg(229653)
True 200
20250609204952_371810a746fd64e5cab0a6f546e5e690_IMAG01_31.jpg
Writing to 20250609204952_371810a746fd64e5cab0a6f546e5e690_IMAG01_31.jpg(111107)
True 200
20250609204952_371810a746fd64e5cab0a6f546e5e690_IMAG01_32.jpg
Writing to 20250609204952_371810a746fd64e5cab0a6f546e5e690_IMAG01_32.jpg(93227)


* 현재 요청된 페이지의 image 모두 다운로드 해보기

In [63]:
import requests
from bs4 import BeautifulSoup
import os

webtoon_url = 'https://comic.naver.com/webtoon/detail?titleId=783053&no=194&week=tue'
req_header = {
    'referer':url,
}
res = requests.get(webtoon_url)
print(res.status_code)
if res.ok:
    soup = BeautifulSoup(res.text, 'html.parser')
    print(len(soup.select("img[src$='.jpg']")))
    #https://image-comic.pstatic.net/webtoon
    print(len(soup.select("img[src^='https://image-comic.pstatic.net/webtoon783053/194/']")))
    print(len(soup.select("img[src*='IMAG01']")))

200
335
0
140


### 3. 파일 업로드 하기
* http://httpbin.org/post 업로드 요청을 할 수 있는 url

In [ ]:
import requests

upload_files = {
    
}
#print(upload_files)

url = 'http://httpbin.org/post'
# file 업로드 하려면 requests의 post 함수에 files 속성을 사용합니다.



200


### 4. 캡챠(이미지) API 호출하기
* urllib 사용
* 1. 캡차 키 발급 요청
  2. 캡차 이미지 요청
  3. 사용자 입력값 검증 요청

In [ ]:
# 캡차 키 발급 요청


In [ ]:
# 캡차 이미지 요청


In [ ]:
#  사용자 입력값 검증 요청



* requests를 사용하는 코드로 변경하기
* [requests docs](https://requests.readthedocs.io/en/latest/user/quickstart/)

In [ ]:
# 사용자 입력값 검증 요청



Error Code: 403


### 5. 블로그 검색하기

In [ ]:
import requests
import pprint

headers = {
    'X-Naver-Client-Id': '',
    'X-Naver-Client-Secret': '',
}

payload = {
    'query': '파이썬',
    'display': 100,
    'sort': 'sim'
}

url = 'https://openapi.naver.com/v1/search/blog.json'


# requests get(url, params, headers) 요청 

# json() 함수로 응답 결과 가져오오기
# 'title' , 'bloggername' , 'description' , 'bloggerlink' , 'link'

# 'data/nhnblog.txt' 파일 생성하기
